In [1]:
import time
import lattice_challenges_01

# in this notebook's comments, "triangular matrix"
#   should always have the meaning of "lower triangular matrix"

In [2]:
# this function returns the coefficient in Gram-Schmidt
#   orthogonalization; the orthogonalized vector is v_i;
#   the vector by which it is improved is v_j
def gs_mu(v_i, v_j):
    return (v_i*v_j)/(v_j*v_j)

# this function returns a matrix which is the result
#   of Gram-Shmidt orthogonalization process (not orthonormalization)
def gram_schmidt_orthogonalization(basis):
    ret = matrix(RR, basis.nrows(), basis.ncols())
    for i in range(basis.nrows()):
        v = basis[i]
        for j in range(i):
            v -= gs_mu(v, ret[j]) * ret[j]
        ret[i] = v
    return ret

# used in LLL (regardless of which condition is used)
def swap_rows_in_LLL(b, g, i):
    g_i_1 = g[i] + gs_mu(b[i], g[i-1])*g[i-1]
    g[i] = g[i-1] - gs_mu(b[i-1], g_i_1)*g_i_1
    g[i-1] = g_i_1
    b[i], b[i-1] = b[i-1], b[i]

# implementation of classical LLL algorithm
# input_basis is the matrix to be reduced;
#   delta should be a number between 0.25 and 0.75 (including 0.75)
#   but the algorithm terminates for delta < 1.0
def LLL_usual_condition(input_basis, delta):
    b = copy(input_basis)
    n = b.nrows()
    g = gram_schmidt_orthogonalization(b)
    i = 0
    while i < n:
        for j in range(i-1, -1, -1):
            b[i] -= round(gs_mu(b[i], g[j])) * b[j]
        if i > 0 and \
                (delta - gs_mu(b[i], g[i-1])**2)*(g[i-1]*g[i-1]) > g[i]*g[i]:
            swap_rows_in_LLL(b, g, i)
            i -= 1
        else:
            i += 1
    return b

# implementation of LLL algorithm with different condition as used in
#   https://link.springer.com/chapter/10.1007/978-3-319-94821-8_10
# input_basis is the matrix to be reduced;
#   delta should be a number between 0.25 and 0.75 (not including 0.75)
def LLL_unusual_condition(input_basis, delta):
    b = copy(input_basis)
    n = b.nrows()
    g = gram_schmidt_orthogonalization(b)
    i = 0
    while i < n:
        for j in range(i-1, -1, -1):
            b[i] -= round(gs_mu(b[i], g[j])) * b[j]
        if i > 0 and delta*(g[i-1]*g[i-1]) > g[i]*g[i]:
            swap_rows_in_LLL(b, g, i)
            i -= 1
        else:
            i += 1
    return b

# returns A * A^T
# input must be integer matrix
def gram_matrix(A):
    ret = matrix(ZZ, A.nrows(), A.nrows())
    for i in range(A.nrows()):
        for j in range(i):
            ret[i, j] = A[i]*A[j]
            ret[j, i] = ret[i, j]
    for k in range(A.nrows()):
        ret[k, k] = A[k]*A[k]
    return ret

def gram_matrix_RR(A):
    ret = matrix(RR, A.nrows(), A.nrows())
    for i in range(A.nrows()):
        for j in range(i):
            ret[i, j] = A[i]*A[j]
            ret[j, i] = ret[i, j]
    for k in range(A.nrows()):
        ret[k, k] = A[k]*A[k]
    return ret

# Cholesky decomposition of G (positive definite quadratic form)
def cholesky_decomposition(G):
    rows = G.nrows()
    ret = matrix(RR, rows, rows)
    for m in range(rows):
        for n in range(m):
            t = 0  # temporary sum
            for i in range(n):
                t += ret[m,i]*ret[n,i]
            ret[m,n] = (G[m,n]-t)/ret[n,n]
        t = 0
        for i in range(m):
            t += ret[m,i]**2
        ret[m,m] = sqrt(G[m,m]-t)
        # the preceding expression can also be taken with negative sign
    return ret

# returns a triangular matrix having the same Gramian matrix
def triangularize_basis(b):
    return cholesky_decomposition(gram_matrix(b))

# this function reduces the input tr_basis from bottom
# performs the same changes on parallel_basis
# the function has side effect on both input parameters
# the return value indicates whether any change of the input
#   parameters has been made (is True if so)
def reduce_triangular_basis(tr_basis, parallel_basis):
    change = False
    for c in range(tr_basis.nrows()-1, -1, -1):
        for m in range(c+1, tr_basis.nrows()):
            t = round(tr_basis[m, c]/tr_basis[c, c])
            if t != 0:
                change = True
                tr_basis[m] -= t*tr_basis[c]
                parallel_basis[m] -= t*parallel_basis[c]
    return change

# similar effect as reduce_triangular_basis above
def reduce_triangular_basis_from_top(tr_basis, parallel_basis):
    change = False
    for m in range(1, tr_basis.nrows()):
        for c in range(m-1, -1, -1):
            t = round(tr_basis[m, c]/tr_basis[c, c])
            if t != 0:
                change = True
                tr_basis[m] -= t*tr_basis[c]
                parallel_basis[m] -= t*parallel_basis[c]
    return change

# returns the sum of natural logarithms of the elements of the input alist
def sum_of_logarithms(alist):
    ret = 0.0
    for val in alist:
        ret += float(log(val))
    return ret

# sorts input b via side effect; returns a list of squared euclidean norms
def sort_by_norms(b):
    squared_norms = [(b[i]*b[i], i) for i in range(b.nrows())]
    squared_norms.sort()
    perm_dict = {}
    for i in range(len(squared_norms)):
        perm_dict[i] = squared_norms[i][1]
    while perm_dict:
        start_index, next_index = perm_dict.popitem()
        start_row = b[start_index]
        cur_index = start_index
        while next_index != start_index:
            b[cur_index] = b[next_index]
            cur_index = next_index
            next_index = perm_dict.pop(next_index)
        b[cur_index] = start_row
    return [squared_norms[i][0] for i in range(b.nrows())]

# sort by euclidean norms and reduce as long
#   as sum of logarithms gets lower
# works with a copy of input_basis; does not have side effects
def reduce_basis_using_triangular_new(input_basis):
    basis = copy(input_basis)
    previous_value = sum_of_logarithms(sort_by_norms(basis))
    while True:
        previous = copy(basis)
        tr_basis = triangularize_basis(basis)
        if not reduce_triangular_basis(tr_basis, basis):
            return previous
        t = sum_of_logarithms(sort_by_norms(basis))
        if t >= previous_value:
            return previous
        previous_value = t

# similar effect as reduce_basis_using_triangular_new above
def reduce_basis_using_triangular_new_from_top(input_basis):
    basis = copy(input_basis)
    previous_value = sum_of_logarithms(sort_by_norms(basis))
    while True:
        previous = copy(basis)
        tr_basis = triangularize_basis(basis)
        if not reduce_triangular_basis_from_top(tr_basis, basis):
            return previous
        t = sum_of_logarithms(sort_by_norms(basis))
        if t >= previous_value:
            return previous
        previous_value = t

# computes some properties of input_matrix using euclidean norm
def euclidean_properties(input_matrix):
    shortest_vector_norm = math.inf
    longest_vector_norm = 0.0
    sum_of_norms = 0.0
    sum_of_logarithms_of_norms = 0.0  # natural log
    for i in range(input_matrix.nrows()):
        t = 0.0
        for j in range(input_matrix.ncols()):
            t += input_matrix[i, j]**2
        r = sqrt(t)
        sum_of_norms += r
        sum_of_logarithms_of_norms += float(log(r))
        if r > longest_vector_norm:
            longest_vector_norm = r
        if r < shortest_vector_norm:
            shortest_vector_norm = r
    return (shortest_vector_norm, longest_vector_norm,
            sum_of_norms, sum_of_logarithms_of_norms)

# sort by euclidean norms and reduce as long
#   as sum of logarithms gets lower
# works with a copy of input_basis; does not have side effects
def reduce_basis_using_triangular_new_A(input_basis):
    B = copy(input_basis)
    old = euclidean_properties(B)[3]*2
    cur = euclidean_properties(B)[3]
    while cur < old:
        old = cur
        B = reduce_basis_using_triangular_new(B)
        B = reduce_basis_using_triangular_new_from_top(B)
        cur = euclidean_properties(B)[3]
    return B

# similar effect as reduce_basis_using_triangular_new_A above
def reduce_basis_using_triangular_new_B(input_basis):
    B = copy(input_basis)
    old = euclidean_properties(B)[3]*2
    cur = euclidean_properties(B)[3]
    while cur < old:
        old = cur
        B = reduce_basis_using_triangular_new_from_top(B)
        B = reduce_basis_using_triangular_new(B)
        cur = euclidean_properties(B)[3]
    return B

# in triangular matrix, this function changes row (vector)
#   at index line_index in input_matrix by subtracting row
#   at index by_index k times
def sub_in_tr(input_matrix, line_index, by_index, k):
    for i in range(by_index+1):
        input_matrix[line_index, i] -= k*input_matrix[by_index, i]

# performs changes in triangular matrix (tr) corresponding to swaping
#   rows at indices i and i-1 in the corresponding basis, so that
#   gramian matrices of tr and the corresponding basis remain the same
def swap_rows_in_tr(tr, i):
    bk = tr[i, i-1]
    ak = tr[i-1, i-1]
    bk1 = tr[i, i]
    tr[i, i-1] = sqrt(bk**2+bk1**2)
    tr[i-1, i-1] = ak*bk/tr[i, i-1]
    tr[i-1, i] = ak*bk1/tr[i, i-1]
    tr[i, i] = 0
    tr[i], tr[i-1] = tr[i-1], tr[i]
    for x in range(i+1, tr.nrows()):
        xa = tr[x, i-1]
        tr[x, i-1] = (xa*bk+tr[x, i]*bk1)/tr[i-1, i-1]
        tr[x, i] = (xa*ak-tr[x, i-1]*tr[i, i-1])/tr[i, i]

# faster implementation of classical LLL
# uses triangular matrix instead of Gram-Schmidt orthogonalization
# does not perform for-cycle after i has been decreased
def LLL_usual_condition_using_triangular(input_basis, delta):
    number_of_vectors = input_basis.nrows()
    basis = copy(input_basis)
    tr = triangularize_basis(basis)
    i = 0
    flag = False
    while i < number_of_vectors:
        if flag:
            for j in range(i-1, -1, -1):
                t = round(tr[i, j]/tr[j, j])
                sub_in_tr(tr, i, j, t)
                basis[i] -= t*basis[j]
        if i>0 and tr[i-1, i-1]**2 * delta > tr[i, i]**2 + tr[i, i-1]**2:
            swap_rows_in_tr(tr, i)
            basis[i], basis[i-1] = basis[i-1], basis[i]
            i -= 1
            flag = False
        else:
            i += 1
            flag = True
    return basis

# faster implementation of LLL with different condition
# uses triangular matrix instead of Gram-Schmidt orthogonalization
# does not perform for-cycle after i has been decreased
def LLL_unusual_condition_using_triangular(input_basis, delta):
    sqrt_delta = sqrt(delta)
    number_of_vectors = input_basis.nrows()
    basis = copy(input_basis)
    tr = triangularize_basis(basis)
    i = 0
    flag = False
    while i < number_of_vectors:
        if flag:
            for j in range(i-1, -1, -1):
                t = round(tr[i, j]/tr[j, j])
                sub_in_tr(tr, i, j, t)
                basis[i] -= t*basis[j]
        if i>0 and tr[i-1, i-1]*sqrt_delta > tr[i, i]:
            swap_rows_in_tr(tr, i)
            basis[i], basis[i-1] = basis[i-1], basis[i]
            i -= 1
            flag = False
        else:
            i += 1
            flag = True
    return basis


In [3]:
# the rank of the input basis must be equal to .nrows()
# assumed the inputs are integer matrix and integer vector
#   of the same dimension (row length)
def check_vector_in_lattice(basis, a_vector):
    return basis.solve_left(a_vector).denominator() == 1

# assumed the inputs are integer matrices of the same dimension
# for matrices with linearly dependant rows the returned value
#   can possibly be False even if tested_matrix is a correct
#   basis defining the same lattice as original_basis
def check_reduction_correctness(original_basis, tested_matrix):
    ret = True
    for a_row in tested_matrix:
        if not check_vector_in_lattice(original_basis, a_row):
            return False
    return original_basis.rank()==tested_matrix.rank()


In [4]:
# adds the_matrix to file as a python function
# creates or modifies a file
def add_matrix_to_file(filename, matrixname, the_matrix):
    f = open(filename, 'a')
    f.write("\ndef "+str(matrixname)+"():\n    return [")
    for i in range(the_matrix.nrows()-1):
        f.write(str(list(the_matrix[i]))+",\n            ")
    if the_matrix.nrows()>0:
        f.write(str(list(the_matrix[the_matrix.nrows()-1])))
    f.write("]\n")
    f.close()


In [5]:
# generates all k-element subsets of {0, 1, ..., n-1} as ordered lists
def k_elem_subsets(n, k):
    indices = list(range(k))
    while True:
        yield indices[:]
        i = -1
        while i>=-k and indices[i]==n+i:
            i -= 1
        if i<-k:
            break
        val = indices[i]
        for j in range(i, 0):
            val += 1
            indices[j] = val

# auxiliary generator used in ceils_and_floors
def ceils_and_floors_aux(alist, acc, c):
    if c==len(alist):
        yield acc[:]
    else:
        acc[c] = floor(alist[c])
        yield from ceils_and_floors_aux(alist, acc, c+1)
        t = ceil(alist[c])
        if t != acc[c]:
            acc[c] = t
            yield from ceils_and_floors_aux(alist, acc, c+1)

# generates all possible combinations (as lists) of ceils and floors
#   of the input seq
def ceils_and_floors(seq):
    alist = list(seq)
    yield from ceils_and_floors_aux(alist, [0]*len(alist), 0)


In [6]:
# G ... Gramian matrix
# index ... index of the reduced row
# indices ... list of indices of rows by which the
#             index-th row is being reduced
# returns a list of non-integer coefficients defining
#   how many times the rows should be added
def gram_and_solve_find_min(G, indices, index):
    d = len(indices)
    left_side = matrix(ZZ, d, d)
    right_side = [0]*d
    for i in range(d):
        for j in range(d):
            left_side[i, j] = G[indices[i], indices[j]]
        right_side[i] = -G[indices[i], index]
    return list(left_side.solve_right(vector(right_side)))

# reduces each row by each possible subset of other rows
#   as long as anything changes
def gram_and_solve_reduction(basis):
    B = copy(basis)
    G, n, k = gram_matrix(B), B.nrows(), 2
    while k<=n:
        reset = False
        for indices in k_elem_subsets(n, k):
            for index in indices:
                alist = gram_and_solve_find_min(G, indices[:index]+indices[index+1:], index)
                min_vec, min_norm, min_comb = vector([0]*B.ncols()), math.inf, vector([0]*k)
                for comb in ceils_and_floors(alist[:index]+[1]+alist[index:]):
                    tvec, avec = vector(comb), vector(ZZ, B.ncols())
                    for i in range(k):
                        avec += B[indices[i]]*tvec[i]
                    t = avec*avec
                    if t < min_norm:
                        min_norm, min_vec, min_comb = t, avec, tvec
                if min_norm < B[index]*B[index]:
                    B[index], bvec = min_vec, vector(ZZ, n)
                    for i in range(k):
                        bvec += G[indices[i]]*min_comb[i]
                    G[index] = bvec
                    for i in range(n):
                        G[i, index] = G[index, i]
                    c = 0
                    for i in range(k):
                        c += G[index, indices[i]]*min_comb[i]
                    G[index, index], k, reset = c, 1, True
                    break
            if reset:
                break
        k += 1
    return B

# reduces each row by the other rows
#   as long as anything changes
def gram_and_solve_simplified(basis):
    B = copy(basis)
    G, n, indices, change = gram_matrix(B), B.nrows(), list(range(B.nrows())), True
    m = n  # m is index of the most recently changed row
    while change:
        change = False
        for index in range(n):
            if index == m:
                break
            alist = gram_and_solve_find_min(G, indices[:index]+indices[index+1:], index)
            min_vec, min_norm, min_comb = vector([0]*B.ncols()), math.inf, vector([0]*n)
            for comb in ceils_and_floors(alist[:index]+[1]+alist[index:]):
                tvec, avec = vector(comb), vector(ZZ, B.ncols())
                for i in range(n):
                    avec += B[i]*tvec[i]
                t = avec*avec
                if t < min_norm:
                    min_norm, min_vec, min_comb = t, avec, tvec
            if min_norm < B[index]*B[index]:
                B[index], bvec = min_vec, vector(ZZ, n)
                for i in range(n):
                    bvec += G[i]*min_comb[i]
                G[index] = bvec
                for i in range(n):
                    G[i, index] = G[index, i]
                c = 0
                for i in range(n):
                    c += G[index, i]*min_comb[i]
                G[index, index], change, m = c, True, index
    return B


In [7]:
# basis ... input basis (should be linearly independent)
# R ... radius (the length of the shortest vector in basis implicitly)
# returns the shortest nonzero vector inside a ball of given radius
def enumeration_SVP_using_triangular(basis, R=None):
    n, G = basis.nrows(), gram_matrix(basis)
    s2, i_cur_min = G[0, 0], 0
    for i in range(1, n):
        if G[i, i] < s2:
            i_cur_min, s2 = i, G[i, i]
    s_comb = [(1 if i==i_cur_min else 0) for i in range(n)]
    if R is None:
        R2 = s2
    else:
        R2 = R*R
        s2 = min(R2, s2)
    b, x, q, p, k, r, t, i = cholesky_decomposition(G), [0]*n, [0.0]*n, [0.0]*n, [0]*n, [0.0]*n, [0]*n, n-1
    inr_bool = [True]*n  # True for k[i] ranging over 0, 1, -1, 2, -2, ...
    while i<n:
        if i==0:
            x[0] = t[0] + k[0]
            q_i_1 = q[0]+(p[0]+b[0, 0]*x[0])**2
            if q_i_1 > 0.0 and q_i_1 < s2:
                s2, s_comb = q_i_1, x[:]
                if inr_bool[0]:
                    k[0] = (-k[0]+1 if k[0]<=0 else -k[0])
                else:
                    k[0] = (-k[0]-1 if k[0]>=0 else -k[0])
                continue
            k[0], i = 0, 1
            continue
        x[i] = t[i] + k[i]
        q[i-1] = q[i]+(p[i]+b[i,i]*x[i])**2
        if q[i-1] <= s2:
            p[i-1] = b[i, i-1]*x[i]+r[i]
            a = -p[i-1]/b[i-1, i-1]
            t[i-1] = round(a)
            inr_bool[i-1] = (t[i-1]==math.floor(a))
            if inr_bool[i]:
                k[i] = (-k[i]+1 if k[i]<=0 else -k[i])
            else:
                k[i] = (-k[i]-1 if k[i]>=0 else -k[i])
            if i>1:
                r[i-1] = sum([b[j, i-2]*x[j] for j in range(i, n)])
            i -= 1
        else:
            k[i] = 0
            i += 1
    ret = vector(s_comb)*basis
    if ret*ret <= R2:
        return ret
    return None


In [8]:
# on this matrix, LLL does not find the shortest vector
#   while gramAndSolve and enumeration do
# (from https://crypto.stackexchange.com/questions/37196/find-an-example-of-a-lattice-such-that-lll-algorithm-cant-find-the-shortest-vec)
L = matrix([[16252, 5541, 48769, 2593, 22395],
            [2404, 46418, 5335, 59631, 29390],
            [41816, 27475, 56363, 50417, 62371],
            [27419, 53783, 1412, 42344, 55983],
            [62503, 52719, 22160, 3940, 64256]])

# on this matrix, the enumeration runs unusually long (even longer than gram_and_solve_simplified)
# found randomly
A = matrix([[39450, -1, -51965, 37376, 54699, 62019, -24917, -56741, -37148, 30791, -61252, 56546],
            [28213, 18923, -39890, -12599, 54760, -29860, 37255, 52830, 56208, 40008, 40911, -28483],
            [-6979, 25394, 28906, -16173, -21753, -62430, 36910, -1379, -62922, 6759, -40726, -1222],
            [51019, 26934, 56685, 5662, 28091, -64399, 3264, 59083, -31035, 49120, -10801, 56806],
            [-34891, -45174, -1956, 52203, 27533, 63705, -62060, -62047, 55565, 58240, 19340, -10976],
            [-9006, -6421, -62272, -62015, 47794, 10658, -27132, -15272, -16756, 48073, 49068, 23670],
            [6825, 49025, 15065, 53684, 57899, -16880, -52560, 43522, 54587, -21981, -64346, -5357],
            [-34387, -16659, -31467, 24034, 59394, -61422, 49348, -33197, 8477, 12763, -10272, -2929],
            [60419, 27900, 35501, -4033, 34752, -37807, -50648, 23311, -13503, 60911, -18204, 46778],
            [45047, 45636, -29536, 14926, -33075, -510, -26142, -4026, -56696, -9264, -25461, 47085],
            [13658, -18002, 51372, -28904, -43333, -47794, 23066, 15399, 37600, 54859, 65516, -51652],
            [51022, 62799, 16837, 12314, -39644, -31050, -54361, 13868, 34833, 63125, -15206, -34825]])

In [14]:
def LLL_usual_condition_using_triangular_with_prints(input_basis, delta):
    number_of_vectors = input_basis.nrows()
    basis = copy(input_basis)
    tr = triangularize_basis(basis)
    last = copy(tr)
    last[0,0] += 1
    i = 0
    flag = True
    while i < number_of_vectors:
        if last != tr:
            print(tr)
            print()
            last = copy(tr)
        if flag:
            for j in range(i-1, -1, -1):
                t = round(tr[i, j]/tr[j, j])
                sub_in_tr(tr, i, j, t)
                basis[i] -= t*basis[j]
        if i>0 and tr[i-1, i-1]**2 * \
                   (delta-(tr[i, i-1]/tr[i-1, i-1])**2) > tr[i, i]**2:
            swap_rows_in_tr(tr, i)
            basis[i], basis[i-1] = basis[i-1], basis[i]
            i -= 1
            flag = False
        else:
            i += 1
            flag = True
    if last != tr:
        print(tr)
        print()
        last = copy(tr)
    return basis

n = 65537
C = matrix([[8*n,0,0,0,0],[0,32*n,0,0,0],[0,0,64*n,0,0],
            [8*(-12), 32*7, 64*11, 1, 0],
            [8*4, 32*(-9), 64*(-5), 0, n]])
LLL_usual_condition_using_triangular_with_prints(C, 0.75)


[  524296.000000000  0.000000000000000  0.000000000000000  0.000000000000000  0.000000000000000]
[ 0.000000000000000 2.09718400000000e6  0.000000000000000  0.000000000000000  0.000000000000000]
[ 0.000000000000000  0.000000000000000 4.19436800000000e6  0.000000000000000  0.000000000000000]
[ -96.0000000000000   224.000000000000   704.000000000000   1.00000000000000  0.000000000000000]
[  32.0000000000000  -288.000000000000  -320.000000000000  0.000000000000000   65537.0000000000]

[  524296.000000000  0.000000000000000  0.000000000000000  0.000000000000000  0.000000000000000]
[ 0.000000000000000 2.09718400000000e6  0.000000000000000  0.000000000000000  0.000000000000000]
[ -96.0000000000000   224.000000000000   704.000710226915  0.000000000000000  0.000000000000000]
[ 0.000000000000000  0.000000000000000 4.19436376853688e6   5957.90308030807  0.000000000000000]
[  32.0000000000000  -288.000000000000  -319.999677169910 -0.454544996022077   65537.0000000000]

[   524296.000000000   0.000

[    -96     224     704       1       0]
[      0       0       0   65537       0]
[    -64     -64     384       1   65537]
[-148008 -178944   36800   17926       0]
[ 367552 -158560  100864   18017       0]